# Rayo: Your daily news help

In [6]:
import requests
import re
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from bs4 import BeautifulSoup
from transformers import pipeline

In [7]:
#A better architecture will be created to select news category later. For now we stick to tech news
default_url = "https://www.bbc.com"
url = "https://www.bbc.com/news/technology"
'''
category = input("Enter a category:")
if category == "technology":
    url = "https://www.bbc.com/news/technology"
elif category == "business":
    url = "https://www.bbc.com/news/business"
elif category == "entertainment":
    url = "https://www.bbc.com/news/entertainment_and_arts"
elif category == "health":
    url = "https://www.bbc.com/news/health"
elif category == 'science':
    url = "https://www.bbc.com/news/science_and_environment"
elif category == 'sports':
    url = "https://www.bbc.com/sport"
elif category == 'world':
    url = "https://www.bbc.com/news/world"

    '''
# Set a time threshold for "recent" news (e.g., up to a day)
one_day_ago = datetime.now() - timedelta(days=1)

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the news articles on the page
    articles = soup.find_all('a',class_='qa-heading-link lx-stream-post__header-link')

    # Initialize lists to store article headlines and links
    news_headlines = []
    news_links = []

    # Loop through the articles and extract relevant information
    for article in articles:
        link = article.find('href')
        " ".join(article.text.split())
        news_headlines.append(article.text)

        #Extract link
        string_soup = str(article).replace('<html><body>', '').replace('</body></html>', '').replace('<p>', '').replace('</p>', '')
        pattern = r'href="([^"]*)"'
        match = re.search(pattern, string_soup)
        if match:
            extracted_url = match.group(1)
            news_links.append(default_url + extracted_url)
        else:
            print("No match found.")
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


In [8]:
content = []

for url in news_links:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        text = ""
        for paragraph in soup.find_all('p'):
            text += paragraph.get_text() + " "
        content.append(text)
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

In [9]:
daily_news_data = pd.DataFrame({'Headline': news_headlines, 'Link': news_links, 'Content': content})

In [10]:
# Create a summarization pipeline using the "facebook/bart-large-cnn" model
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [11]:
def generate_summary(content):
    summary = pipe(content, max_length=150, min_length=50, do_sample=False)
    summarized_text = summary[0]['summary_text']
    return summarized_text

In [12]:
summarized_text = generate_summary(daily_news_data['Content'][0])
print (summarized_text)

Tinder Matchmaker feature allows friends and family to recommend matches. Up to 15 people can view a profile without needing to log in to Tinder. Dating expert Sarah Louise Ryan says there could be privacy concerns. The feature will first be rolled out in 15 countries, including the UK, the US and Australia.
